## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [4]:
import glob

import numpy as np
import scipy as sp
import scipy.io
from scipy import io, signal
from matplotlib import pyplot as plt
from matplotlib import mlab

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    data
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        print ("first file")
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        print (errors)
        print (confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

#def RunPulseRateAlgorithm(data_fl, ref_fl):
    # Load data using LoadTroikaDataFile
#    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    
    # Compute pulse rate estimates and estimation confidence.

    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
#    errors, confidence = np.ones(100), np.ones(100)  # Dummy placeholders. Remove
#    return errors, confidence

def RunPulseRateAlgorithm(data_fl, ref_fl): 
    """ Calculates mean absolute errors and confidence values
    
    Args: 
        data_fl: (str) filepath to a troika .mat file.
        ref_f1: (str) filepath to a troika .mat file which contains reference heart beat estimates.
        
    Returns:
        error_array: Array of absolute errors
        conf_array: Array of confidence values for each error value.
    """
        
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    # Loading the reference file
    ground_truth = scipy.io.loadmat(ref_fl)['BPM0'].reshape(-1)
    
    # Bandpass ppg and acc signals 
    ppg = bandpass_filter(ppg)
    accx = bandpass_filter(accx)
    accy = bandpass_filter(accy)
    accz = bandpass_filter(accz)
        
    # Calculate Power Spectral Density estimates for each signal
    psd_ppg, freqs_ppg = cal_spectogram(signal=ppg)
    psd_accx, freqs_accx = cal_spectogram(signal=accx)
    psd_accy, freqs_accy = cal_spectogram(signal=accy)
    psd_accz, freqs_accz = cal_spectogram(signal=accz)
    
    # Get frequencies and time steps
    freqs = freqs_ppg.shape[0]
    time_steps = psd_ppg.shape[1]
    
    # Sorted index value for signals 
    ppg_index = (-psd_ppg).argsort(axis=0)
    accx_index = (-psd_accx).argsort(axis=0)
    accy_index = (-psd_accy).argsort(axis=0)
    accz_index = (-psd_accz).argsort(axis=0)
    estimated_freq = []
    for t in range(time_steps):
        for freq in range(freqs):
            i=0
            if freq == 2:
                estimated_freq.append(freqs_ppg[ppg_index[freq][t]])
                break
            elif np.all([(freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accx_index[i][t]]), 
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accy_index[i][t]]), 
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accz_index[i][t]]),
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accx_index[i+1][t]]),
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accy_index[i+1][t]]),
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accz_index[i+1][t]]),
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accx_index[i+2][t]]),
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accy_index[i+2][t]]),
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accz_index[i+2][t]])]):
                estimated_freq.append(freqs_ppg[ppg_index[freq][t]])
                break
            
    # Claculate confidence
    confidence = []
    for i in range(len(estimated_freq)):
        snr = calc_snr(ppg, estimated_freq[i])
        confidence.append(snr)
    
    pre = np.array(estimated_freq) * 60
    
    # Claculate the absolute error 
    error_array = np.abs(ground_truth - pre)
    conf_array = np.array(confidence)
    # Return per-estimate absolute error and confidence as a 2-tuple of numpy arrays.
    return error_array, conf_array

def bandpass_filter(signal):
    """
    Filter the signal between the minfreq and maxfreq
    
    Args: 
        signal: signal from PPG or Accelerometer
    
    Returns:
        ndarray
  
    """
    pass_band=(minfreq, maxfreq)
    b, a = scipy.signal.butter(3, pass_band, btype='bandpass', fs=fs)
    return scipy.signal.filtfilt(b, a, signal)

def calc_snr(signal, hr_f):
    """
    Calculates the signal to noise ratio
    
    Args:
        signal: signal from PPG
        hr_f: heart rate frequency estimates
        
    Returns:
        int
    
    """
    n = len(signal)*2
    harmonic_f = hr_f * 2
    fft_mag = np.abs(np.fft.rfft(signal, n))
    freqs = np.fft.rfftfreq(n, 1/fs)
    window_f = 5/60
    fundamental_frequency_window = (freqs > hr_f - window_f) & (freqs < hr_f + window_f)
    harmonic_frequency_window = (freqs > harmonic_f - window_f) & (freqs < harmonic_f + window_f)
    signal = np.sum(fft_mag[(fundamental_frequency_window) | (harmonic_frequency_window)])
    noise = np.sum(fft_mag[~ ((fundamental_frequency_window) | (harmonic_frequency_window))])
    snr = signal / noise
    
    return snr
    
    
def cal_spectogram(signal):
    """
    Calculate PSD estimates
    
    Args:
        signal: PPG or Accelerometer signals
    Returns:
        psd: ndarray
        freqs:  ndarray
    
    """
    psd, freqs, t = mlab.specgram(signal, NFFT=8*fs, Fs=fs, noverlap=6*fs, pad_to=12*fs)
    psd = psd[(freqs >= minfreq) & (freqs <= maxfreq)]
    freqs = freqs[(freqs >= minfreq) & (freqs <= maxfreq)]
    
    return psd, freqs

In [5]:
def LowpassFilter(signal, passband):
  b, a = sp.signal.butter(3, passband, btype='bandpass', fs=125)
  return sp.signal.filtfilt(b, a, signal)

In [6]:
fs = 125 # The data set documentation says all signals were sampled at 125 HZ
minfreq=0.67
maxfreq=4.0
print ("hello1")
error = Evaluate()
print ("hello2")
print ("Error =", error)

hello1
first file
[ 70.66079295   1.35746606   2.85714286   0.33185841   2.58064516
   1.68458781   2.10583153   6.55059848   0.33482143   1.8442623
   5.4009434    0.88607595   0.49668874   0.83825266   9.58762887
   6.94244604  12.31958763  10.29978587   2.12707182   1.48106904
   9.65011287  76.76470588  54.04079383  57.12074303  70.26315789
  52.71226415  50.33222591  73.41986456  62.15189873  61.55172414
  61.82539683  47.04329461  36.71610169  45.94450374  53.79084967
  51.41675284  39.33030647  53.41657811  52.70053476  47.21804511
  31.36363636  50.49618321  34.86842105  43.97790055  32.94314381
   1.45416228   0.19230769   1.8851571    1.03305785   0.78616352
   1.93169691   0.86150491   1.81465039  19.50584485  37.80487805
  26.11111111  39.42668137  28.34207765  42.29386892  26.99152542
  21.68789809  36.53560043  36.23003195  20.76923077  35.18292683
  24.29447853  23.09278351  21.7169615   20.15772871  18.72611465
  32.59358289  63.72168285  30.27687296  65.22875817  19.55

In [ ]:
fs = 125 # The data set documentation says all signals were sampled at 125 HZ
data_fls, ref_fls = LoadTroikaDataset()
errs, confs = [], []
data_fl = data_fls[2]
ref_fl = ref_fls[2]

In [ ]:
#ref = LoadTroikaDataFile(ref_fl)
#ref

In [ ]:
ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
print(ppg)
print(len(ppg))
ts = np.arange(0, len(ppg)/fs, 1/fs)
print(np.mean(ppg))
pks=sp.signal.find_peaks(ppg)[0]
        
plt.clf()
plt.figure(0)
plt.title('Time domain')
plt.plot(ppg)
plt.xlabel('Time (sec)')

plt.plot(pks, ppg[pks],"r.",ms=10)                        

In [ ]:
60/(np.diff(pks)/fs)

In [ ]:
mean_pulse_rate = np.mean(60/(np.diff(pks)/fs))
print ("mean pulse rate from time domain =", mean_pulse_rate)

In [ ]:
pks=sp.signal.find_peaks(ppg, height=15, distance=35)[0]

In [ ]:
60/(np.diff(pks)/fs)

In [ ]:
mean_pulse_rate = np.mean(60/(np.diff(pks)/fs))
print ("mean pulse rate from time domain =", mean_pulse_rate)

In [ ]:
freqs = np.fft.rfftfreq(len(ppg), 1/fs)
fft_mag = np.abs(np.fft.rfft(ppg))
plt.figure(1)
plt.title('Frequency domain')
plt.xlabel('Frequency (HZ)')
plt.xlim (0,5)
plt.plot(freqs, fft_mag)

In [ ]:
freqs

In [ ]:
plt.figure(2)
bandpassed_ppg = LowpassFilter(ppg, (0.67, 4)) # Corresponding to 40 HZ to 240 HZ band 

In [ ]:
freqs_ppg = np.fft.rfftfreq(len(bandpassed_ppg), 1/fs)
fft_ppg = np.abs(np.fft.rfft(bandpassed_ppg))
plt.figure(1)
plt.title('Frequency domain (PPG)')
plt.xlabel('Frequency (HZ)')
plt.xlim (0, 4)
plt.plot(freqs_ppg, fft_ppg)

In [ ]:
np.argmax(freqs_ppg)

In [ ]:
freqs_ppg

In [ ]:
freqs_ppg[np.where(fft_ppg == np.amax(fft_ppg))][0]

In [ ]:
freqs.shape

In [ ]:
fft_ppg.shape

In [ ]:
# Splits of 2-second windows since ground truth slides every 2 seconds, so NFFT=fs*8, fs being 125 HZ in our case
spec_ppg, freqs_ppg, _ , _ = plt.specgram(bandpassed_ppg, Fs=fs, NFFT=fs*8, 
                                              noverlap = 6*fs, xextent = [0, len(bandpassed_ppg)/fs]);
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')
plt.title("Bandpass filtered PPG signal")

In [ ]:
spec_ppg.shape

In [ ]:
len(freqs_ppg)

In [ ]:
spec_ppg

In [ ]:
dominant_peaks = freqs_ppg[np.argmax(spec_ppg, axis=0)]
print ("Dominant Frequency", dominant_peaks)

In [ ]:
# Histogram of the dominant peaks
a, b, _ = plt.hist(dominant_peaks)

In [ ]:
a

In [ ]:
result = np.where(a == np.amax(a))
a[result[0]][0]

In [ ]:
a.sort()
a

In [ ]:
for i in range(int(np.trunc(len(a)/3))):
    print (a[i*3:i*3+3])

In [ ]:
int(np.trunc(len(fft_ppg)/(fs*2)))-1

In [ ]:
# Corresponding to 40 HZ to 240 HZ band 
accx = LowpassFilter(accx, (0.67, 4))
accy = LowpassFilter(accy, (0.67, 4))
accz = LowpassFilter(accz, (0.67, 4))

acc_mag = np.sqrt(np.sum(np.square(np.vstack((accx, accy, accz))), axis=0))
#acc_mag = LowpassFilter(acc_mag, (0.67, 4))

# Transform signal into frequency domain
freqs = np.fft.rfftfreq(len(acc_mag), 1/fs)
fft_mag = np.abs(np.fft.rfft(acc_mag))
plt.figure(1)
plt.title('Frequency domain (Accelerometer)')
plt.xlabel('Frequency (HZ)')
plt.xlim (0, 4)
plt.plot(freqs, fft_mag)

In [ ]:
freqs

In [ ]:
ground_truth = sp.io.loadmat(ref_fl)['BPM0'].reshape(-1)
ground_truth

In [ ]:
for i in range (int(np.trunc(len(fft_ppg)/(fs*2)))):
#for i in range (1):
    print (i)
    ppg_slice = fft_ppg[i*fs*2:(i+1)*fs*2]
    # Pass only the 40 HZ to 240 HZ band
    bandpassed_slice = LowpassFilter(ppg_slice, (0.67, 4)) 
    #print(len(bandpassed_slice))
    
    # Transform to frequency domain
    freqs_slice = np.fft.rfftfreq(len(bandpassed_slice)*2, 1/fs)
    fft_slice = np.abs(np.fft.rfft(bandpassed_slice, len(bandpassed_slice)*2))

    # Find Peak
    ppg_pks = sp.signal.find_peaks(fft_slice)[0]
    ppg_freqs_pks = np.ones(len(ppg_pks))
    
    #print ("Signal Freq",freqs_slice)
    #print ("Signal Mag", fft_slice[ppg_pks])
    #print ("Signal Peaks (Index to Mag)", ppg_pks)
    
    for j in range(len(ppg_pks)):
        ppg_freqs_pks[j] = freqs_slice[ppg_pks[j]]
        ppg_pks[j] = fft_slice[ppg_pks[j]]
    #print ("Peak Frequencies", ppg_freqs_pks, len(freqs))
    #print ("Peak Magnitudes", ppg_pks)
    
    ppg_pks_sorted = np.sort(ppg_pks)
    ppg_pks_first  = ppg_pks_sorted[-1]
    ppg_pks_second = ppg_pks_sorted[-2]
    ppg_pks_first_freq = ppg_freqs_pks[np.where(ppg_pks == ppg_pks_first)[0][0]]
    ppg_pks_second_freq = ppg_freqs_pks[np.where(ppg_pks == ppg_pks_second)[0][0]]
  
    print ("Highest PPG Peak =", ppg_pks_first, ppg_pks_first_freq)
    print ("Second PPG Peak =", ppg_pks_second, ppg_pks_second_freq)
    
    #peakval = np.max(ppg_pks)
    #print ("Highest Peak =", peakval)
    #print ("Highest Peak Frequency =", ppg_freqs_pks[np.where(ppg_pks == peakval)[0][0]])
    
    plt.figure(i)
    plt.title('Frequency domain (PPG)')
    plt.xlabel('Frequency (HZ)')
    plt.xlim (0, 10)
    #plt.plot(freqs_slice, fft_slice)
    #plt.plot(ppg_freqs_pks, ppg_pks,"r.",ms=10)
    
    # ACC_MAG
    #print (i)
    acc_slice = acc_mag[i*fs*2:(i+1)*fs*2]
    # Pass only the 40 HZ to 240 HZ band
    bandpassed_slice = LowpassFilter(acc_slice, (0.67, 4)) 
    #print(len(bandpassed_slice))
    
    # Transform to frequency domain
    # Multiply length by 4 to maintain the frequency granularity. Else
    # accuracy drops and comparison withe ACC frequencies become meaningless
    freqs_slice = np.fft.rfftfreq(len(bandpassed_slice)*2, 1/fs)
    fft_slice = np.abs(np.fft.rfft(bandpassed_slice, len(bandpassed_slice)*2))

    # Find Peak
    acc_pks = sp.signal.find_peaks(fft_slice)[0]
    acc_freqs_pks = np.ones(len(acc_pks))
    
    #print ("ACC Signal Frequencies =", freqs_slice)
    #print ("ACC Signal Magnitudes=", fft_slice[acc_pks])
    #print ("ACC Peaks (Indices) =", acc_pks,"# of peaks =", len(acc_pks))
  
    for j in range(len(acc_pks)):
        acc_freqs_pks[j] = freqs_slice[acc_pks[j]]
        acc_pks[j] = fft_slice[acc_pks[j]]
    
    # Sort frequency peaks from Accelerometer Magnitude slices
    # acc_freqs_pks = np.sort(acc_freqs_pks)
    
    #print ("ACC Peaks (Frequencies)=",acc_freqs_pks)
    #print ("ACC Peaks (Magnitudes) =",acc_pks)
    
    acc_pks_sorted = np.sort(acc_pks)
    acc_pks_first  = acc_pks_sorted[-1]
    acc_pks_second = acc_pks_sorted[-2]
    acc_pks_first_freq = acc_freqs_pks[np.where(acc_pks == acc_pks_first)[0][0]]
    acc_pks_second_freq = acc_freqs_pks[np.where(acc_pks == acc_pks_second)[0][0]]
  
    print ("Highest ACC Peak =", acc_pks_first, acc_pks_first_freq)
    print ("Second ACC Peak =", acc_pks_second, acc_pks_second_freq)
    
    #peakval = np.max(acc_pks)
    #print ("Highest Peak =", peakval)
    #print ("Highest Peak Frequency =", acc_freqs_pks[np.where(acc_pks == peakval)[0][0]])
    
    plt.figure(i+1)
    plt.title('Frequency domain (ACC)')
    plt.xlabel('Frequency (HZ)')
    plt.xlim (0, 10)
    #plt.plot(freqs_slice, fft_slice)
    #plt.plot(acc_freqs_pks, acc_pks,"r.",ms=10)
   
    # The ground truth for this slice
    ground_truth_slice = ground_truth[i]/60
    print ("Ground Truth = ", ground_truth_slice)
    

In [ ]:
#pks=sp.signal.find_peaks(acc_mag, height=1, distance=15)[0]
        
plt.clf()
plt.figure(0)
plt.title('Time domain (Accelerometer)')
plt.plot(acc_mag)
plt.xlabel('Time (sec)')

#plt.plot(pks, acc_mag[pks],"r.",ms=10)

In [ ]:
# Note that the mean of the dominant BPMs that we get in the frequency domain after band pass filtering (149) is close 
# to what we got in the time domain earlier with basic 'find_peaks filtering' for height/width (155). So we 
# are on the right track

In [ ]:
plt.figure(1)
plt.title('Ground Truth')
plt.ylabel('BPM')
#plt.xlim (0, 4)
plt.plot(ground_truth)

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

In [8]:
"""
This project seeks to determine the heart rate from PPG signals captured from a wearable and corroborate/tune 
results with the help of data elicited from an accelerometer. 

The algorithm slices each PPG dataset into chunks of 2-seconds and slides that window until the end of the signal
For each slice, it performs transformation into frequency domain using Fourier Transforms. It then attenuates it using
a band pass filter by cancelling out frequencies outside of 0.67 HZ (corresponding to 40 BPM) and 
4 HZ (corresponding ot 240 BPM) It then finds out the peaks in the signal. We check that the distance between 
peaks in the time domain roughly corresponds to the highest frequency 
component in the frequency domain.

It then computes the L2 Norm of the accelerometer signals after passing the components through the same band pass
filter as alluded to above. 

If the highest peak in the PPG signal is near to a peak in the accelerometer signal, the next non-overlapping peak
in the PPG signal is chosen.

Error components are then calculated to reach a degree of confidence in the results.
"""

'\nThis project seeks to determine the heart rate from PPG signals captured from a wearable and corroborate/tune \nresults with the help of data elicited from an accelerometer. \n\nThe algorithm slices each PPG dataset into chunks of 2-seconds and slides that window until the end of the signal\nFor each slice, it performs transformation into frequency domain using Fourier Transforms. It then attenuates it using\na band pass filter by cancelling out frequencies outside of 0.67 HZ (corresponding to 40 BPM) and \n4 HZ (corresponding ot 240 BPM) It then finds out the peaks in the signal. We check that the distance between \npeaks in the time domain roughly corresponds to the highest frequency \ncomponent in the frequency domain.\n\nIt then computes the L2 Norm of the accelerometer signals after passing the components through the same band pass\nfilter as alluded to above. \n\nIf the highest peak in the PPG signal is near to a peak in the accelerometer signal, the next non-overlapping peak\

-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 